In [1]:
from ldn.utils import execute, get_job_status, get_tiles
from ldn.processor import LDNProcessor
import s3fs

import time

In [8]:
# Set up variables here.
restart = True
jobs={}

In [ ]:
from IPython.display import clear_output

job_status={}

for year in range(2000, 2024):
    if restart:
        job_id = execute(year)
        jobs[year] = job_id
    job_status[job_id] = get_job_status(job_id)

restart = False

# status = get_job_status(job_id)
not_complete = True
while not_complete:
    clear_output(wait=True)
    for year, job_id in jobs.items():        # If all jobs are either complete or failed, then break the loop
        if all([status in ["SUCCEEDED", "FAILED"] for status in job_status.values()]):
            print(f"All tasks completed, or failed: {job_status}")
            not_complete = False
            break
    
        status = get_job_status(job_id)
        job_status[job_id] = status

        print(f"{year} is {status}")

    if not_complete:
        time.sleep(2)

In [ ]:

version = "0.1.0"

path = f"s3://data.ldn.auspatious.com/geo_ls_lp/{version.replace('.', '_')}/**/*.stac-item.json"

fs = s3fs.S3FileSystem(anon=True)
stac_keys = fs.glob(path)

print(f"Found {len(stac_keys)} out of {30*(2024-2000)} STAC keys like this: {stac_keys[0]}")

incomplete_tiles = []
tiles = get_tiles()

for year in range(2000, 2024):
    for tile, geobox in tiles:
        processor = LDNProcessor(
            tile,
            year=year,
            bucket="data.ldn.auspatious.com",
            configure_s3=False,
            version="0.1.0"
        )
        key = f"{processor.bucket}/{processor.stac_key}"
        if not key in stac_keys:
            incomplete_tiles.append((tile, year))
            print(f"Tile {tile} from {year} is not complete yet")
            

In [ ]:
for tile, year in incomplete_tiles:
    job_id = execute(year, tile=tile)